In [ ]:
import numpy as np
import psana
import matplotlib.pyplot as plt
import time
import random

In [ ]:
experiment='cxilw1118'
runNumber=30

import os
os.environ['INSTALLPATH']= '/cds/home/i/igabalsk/TRXS-Run18'
os.environ['OUTPUTPATH']= '/cds/data/psdm/%s/%s/scratch' % (experiment[0:3],experiment)

In [ ]:
# Magic iPython command to enable plotting
%matplotlib inline

# Load in the pythonBatchMagic library
import sys
sys.path.insert(0, os.environ['INSTALLPATH']+'/Libraries/pythonBatchMagic')
from pythonBatchMagic import *

# Determine current user
currentUser, error = unixCMD("echo $USER")
currentUser = currentUser.strip()
print(currentUser)
os.environ['RESULTSPATH']= ('/cds/data/psdm/%s/%s/results/%s' % (experiment[0:3],experiment,currentUser)).strip()
if not os.path.exists(os.environ['RESULTSPATH']):
    os.mkdir(os.environ['RESULTSPATH'])

sys.path.insert(0, os.environ['INSTALLPATH']+'/Libraries/LCLS')
from LCLSdefault import *

sys.path.insert(0, os.environ['INSTALLPATH']+'/Libraries/mattsLibraries')
from picklez import *

# Load in the get data library
from dataAnalysis import *

# Load in the batch library for lcls
from lclsBatch import *

In [ ]:
detDict = {
    'uvint':{'name': 'Acqiris','get-function':'getAcqirisSum2'},
           'uvint1':{'name': 'Acqiris','get-function':'getAcqirisSum1'},
           'gasPressure':{'name': 'CXI:MKS670:READINGGET','get-function':'getGasPressure'},
           'xrayEnergy':{'name':'FEEGasDetEnergy','get-function': 'getXrayEnergy' },
           'ebeamcharge':{'name':'EBeam', 'get-function': 'getEbeamCharge'},
          'stageencoder':{'name':'CXI:LAS:MMN:04.RBV','get-function':'getStageEncoder'},
          'ttfltpos':{'name':'CXI:TIMETOOL:FLTPOS', 'get-function': 'getTTFltPos'},
#           'vitpos':{'name':'LAS:FS5:VIT:FS_TGT_TIME_DIAL', 'get-function': 'getTTFltPos'},
#            'xint':{'name':'CxiDg3_Imb01','get-function': 'getIPM' },
#            'xpos':{'name':'CxiDg3_Imb01','get-function': 'getXPos' },
#            'ypos':{'name':'CxiDg3_Imb01','get-function': 'getYPos' },
          'seconds':{'name':'seconds', 'get-function': 'getSeconds'},
          'nanoseconds':{'name':'nanonseconds', 'get-function': 'getNanoseconds'},
          'fiducials':{'name':'fiducials', 'get-function': 'getFiducials'},
            'cspad':{'name':'jungfrau4M','get-function':'getCSPAD'}
#           'rois':{'name':'jungfrau4M', 'get-function': 'getRadialrois'},
#           'laserOn':{'name':'evr1', 'get-function': 'getLaserOn'},
#           'xrayOn':{'name':'evr1', 'get-function': 'getXrayOn'},
#           'photonHist':{'name':'jungfrau4M', 'get-function': 'getPhotonHistogram'}
          }

print 'exp=%s:run=%d:smd' % (experiment , runNumber) 
ds = psana.DataSource('exp=%s:run=%d:smd' % (experiment , runNumber) )
eventMax = 50000
events = ds.events()


### Grab some images

In [ ]:
n_images = 30
images = np.zeros((8,512,1024,n_images))
jf = psana.Detector('jungfrau4M')
tt = psana.Detector('CXI:TIMETOOL:FLTPOS')
dg3 = Detector('CXI-DG3-BMMON')
dg3_vals = []
n_filled = 0
for i, evt in enumerate(ds.events()):
    if n_filled==n_images:
        break
    evtId = evt.get(EventId)
    seconds = evtId.time()[0]
    tt_val = tt(evt)
#     print seconds, tt_val
    if i%10==0:
        print i
    image = jf.calib(evt)
    dg3_event = dg3.get(evt)
    if dg3_event is None:
        print 'its none'
        continue
    xint = np.abs(dg3_event.TotalIntensity())
    print 'xint',xint
    if xint<11000:
        continue
#     if image is not None:
#         print 'good at',i
    image[(image<0)|(image>1000)]=0
    images[:,:,:,n_filled] = image
    dg3_vals.append(xint)
    n_filled += 1
print images.shape

In [ ]:
images_thres = np.copy(images)
images_thres[(images_thres<100)|(images_thres>1000)]=0
x,y = CSPADgeometry(detType='Jungfrau', experiment=experiment, run=runNumber)
cspadMask = np.ones_like(x).astype(bool)
plotCSPAD(np.sum(images_thres,axis=-1),x,y, cspadMask=cspadMask,zLims=[0,500])
# savefig('dark_jungfrau.png')

In [ ]:
one_photon = 600
hists, bins = np.histogram(images[(images>100)&(images<1000)],bins=1000)
num_photons = np.sum(hists*bins[:-1]/one_photon)/n_images
plt.figure()
plt.plot(bins[:-1],hists*bins[:-1]/one_photon/n_images,label='%.0f photons/image' % num_photons)
plt.xlabel('Pixel value (%d=1 xray)' % one_photon)
plt.ylabel('Counts')
plt.title('%s Run %d, mean(xint)=%.3f' % (experiment, runNumber,np.mean(dg3_vals)))
plt.legend()
plt.savefig('%s-run-%d_photonhist.png' % (experiment, runNumber))
plt.show()
# This should be the number of X-rays that hit per image

### Histogram each tile over all grabbed images

In [ ]:
from scipy.ndimage import gaussian_filter

tile_smoothed_hists = []
tile_bins = []
plt.figure(figsize=(9,6))
# images[images>1000]=0
for itile in range(8):
    avg_pixel_val = np.sum(images[itile])/(512.*1024.*n_images)
    print 'Average pixel value: ',avg_pixel_val
    hists, bins = np.histogram(images[itile],bins=500)
    smoothed_hists = gaussian_filter(hists,sigma=2)
#     smoothed_hists[smoothed_hists<10]=0
    plt.semilogy(bins[:-1],smoothed_hists, label='Tile %d' %itile, linewidth=3)
    tile_smoothed_hists.append(smoothed_hists)
    tile_bins.append(bins[:-1])
plt.legend(fontsize=15)
plt.rc('font',size=20)
ax = plt.gca()
ax.tick_params(which='major',width=2, length=8)
ax.tick_params(which='minor',width=1, length=5)
plt.xlabel('Pixel Value (adu)')
plt.ylabel('Counts')
plt.title('Tile Histograms',fontsize=20)
# plt.savefig('tile_histograms_dark.png')
plt.show()

### Find peaks for each tile

In [ ]:
from scipy.signal import find_peaks
for itile in range(8):
    peaks, properties = find_peaks(tile_smoothed_hists[itile])
    peaks_pixelvals = tile_bins[itile][peaks]
    print peaks_pixelvals[(peaks_pixelvals>550)&(peaks_pixelvals<700)], tile_smoothed_hists[itile][peaks]

### Which tiles have fewer X-ray hits? (the outer ones)

In [ ]:
plt.figure(figsize=(9,6))
plt.semilogy(tile_bins[0],tile_smoothed_hists[0], '--',label='Tile 0',linewidth=3)
plt.semilogy(tile_bins[1],tile_smoothed_hists[1], label='Tile 1',linewidth=3)
plt.semilogy(tile_bins[2],tile_smoothed_hists[2], label='Tile 2',linewidth=3)
plt.semilogy(tile_bins[3],tile_smoothed_hists[3], '--',label='Tile 3',linewidth=3)
plt.semilogy(tile_bins[4],tile_smoothed_hists[4], '--',label='Tile 4',linewidth=3)
plt.semilogy(tile_bins[5],tile_smoothed_hists[5], label='Tile 5',linewidth=3)
plt.semilogy(tile_bins[6],tile_smoothed_hists[6],label='Tile 6',linewidth=3)
plt.semilogy(tile_bins[7],tile_smoothed_hists[7], '--',label='Tile 7',linewidth=3)
plt.legend(fontsize=15)
plt.rc('font',size=20)
ax = plt.gca()
ax.tick_params(which='major',width=2, length=8)
ax.tick_params(which='minor',width=1, length=5)
plt.xlabel('Pixel Value (adu)')
plt.ylabel('Counts')
plt.title('Tile Histograms')
plt.show()

### Does an x-ray hit leak into the adjacent pixels?

In [ ]:
print np.mean(images)

In [ ]:
images_thresh = np.copy(images)
images_thresh[images_thresh<50]=0
images_unthresh = np.copy(images)

In [ ]:
center_val = 300
hit_indices = np.argwhere((images>center_val-10)&(images<center_val+10))
print hit_indices.shape

In [ ]:
meanvals = []
allvals = []
sumvals = []
step = 1
images = images_unthresh
for index in hit_indices:
    if index[1]<step or index[1]>510-step or index[2]<step or index[2]>1022-step:
        continue
    x1 = tuple(index + np.array([0,step,0,0]))
    x2 = tuple(index - np.array([0,step,0,0]))
    y1 = tuple(index + np.array([0,0,step,0]))
    y2 = tuple(index - np.array([0,0,step,0]))
    x1y1 = tuple(index + np.array([0,step,step,0]))
    x1y2 = tuple(index + np.array([0,step,-step,0]))
    x2y1 = tuple(index + np.array([0,-step,step,0]))
    x2y2 = tuple(index + np.array([0,-step,-step,0]))
    
    meanval = (images[x1]+images[x2]+images[y1]+images[y2])/4
    allvals.append(images[x1])
    allvals.append(images[x2])
    allvals.append(images[y1])
    allvals.append(images[y2])
    allvals.append(images[x1y1])
    allvals.append(images[x1y2])
    allvals.append(images[x2y1])
    allvals.append(images[x2y2])
    sumvals.append(images[x1]+images[x2]+images[y1]+images[y2]
                  +images[x1y1]+images[x1y2]+images[x2y1]+images[x2y2]
                  +images[tuple(index)])
    meanvals.append(meanval)
print np.mean(meanvals)
exec('allvals%d = allvals' %step)

meanvals = []
allvals = []
step = 2
for index in hit_indices:
    if index[1]<step or index[1]>510-step or index[2]<step or index[2]>1022-step:
        continue
    x1 = tuple(index + np.array([0,step,0,0]))
    x2 = tuple(index - np.array([0,step,0,0]))
    y1 = tuple(index + np.array([0,0,step,0]))
    y2 = tuple(index - np.array([0,0,step,0]))
    x1y1 = tuple(index + np.array([0,step,step,0]))
    x1y2 = tuple(index + np.array([0,step,-step,0]))
    x2y1 = tuple(index + np.array([0,-step,step,0]))
    x2y2 = tuple(index + np.array([0,-step,-step,0]))
    
    meanval = (images[x1]+images[x2]+images[y1]+images[y2])/4
    allvals.append(images[x1])
    allvals.append(images[x2])
    allvals.append(images[y1])
    allvals.append(images[y2])
    allvals.append(images[x1y1])
    allvals.append(images[x1y2])
    allvals.append(images[x2y1])
    allvals.append(images[x2y2])
    meanvals.append(meanval)
print np.mean(meanvals)
exec('allvals%d = allvals' %step)

In [ ]:
print len(allvals2)

In [ ]:
allhists1, allbins1 = np.histogram(allvals1,bins=100)
allhists2, allbins2 = np.histogram(allvals2,bins=100)
plt.figure(figsize=(9,6))
plt.semilogy(allbins1[:-1],allhists1,label='Step=1',linewidth=3)
plt.semilogy(allbins2[:-1],allhists2,label='Step=2',linewidth=3)
plt.legend()
ax = plt.gca()
ax.tick_params(which='major',width=2, length=8)
ax.tick_params(which='minor',width=1, length=5)
plt.xlabel('Pixel Value (adu)')
plt.ylabel('Counts')
plt.title(r'Pixels adjacent to %d $\pm$ 10 adu'%center_val)
plt.show()

In [ ]:
sumhists, sumbins = np.histogram(sumvals,bins=1000)
smoothed_sumhists = gaussian_filter(sumhists,sigma=5)
sumpeaks, properties = find_peaks(smoothed_sumhists)
print 'Peak locations (with smoothing): ',sumbins[sumpeaks]
plt.figure(figsize=(9,6))
plt.semilogy(sumbins[:-1],sumhists,label='Sum',linewidth=2.5)
plt.legend()
ax = plt.gca()
ax.tick_params(which='major',width=2, length=8)
ax.tick_params(which='minor',width=1, length=5)
plt.xlabel('Pixel Sum (adu)')
plt.ylabel('Counts')
plt.title(r'Sum around %d $\pm$ 10 adu'%center_val)
plt.show()